In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import urllib
import cv2
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
df= pd.read_csv("C:\\Users\\focam\\OneDrive\\Escritorio\\xd\\data.csv")
df

,img,stage,discharge
0,file:///D:/x/StateLineWeir_20120609_Farrell_00...,2.99,916.0
1,file:///D:/x/StateLineWeir_20120609_Farrell_00...,2.99,916.0
2,file:///D:/x/StateLineWeir_20120609_Farrell_00...,2.96,873.0
3,file:///D:/x/StateLineWeir_20120609_Farrell_00...,2.94,846.0
4,file:///D:/x/StateLineWeir_20120609_Farrell_00...,2.94,846.0
...,...,...,...
41289,file:///D:/x/StateLineWeir_20191011_Farrell_40...,2.54,434.0
41290,file:///D:/x/StateLineWeir_20191011_Farrell_41...,2.54,434.0
41291,file:///D:/x/StateLineWeir_20191011_Farrell_41...,2.54,434.0
41292,file:///D:/x/StateLineWeir_20191011_Farrell_41...,2.54,434.0


In [ ]:
img_array = np.load('C:\\Users\\focam\\OneDrive\\Escritorio\\xd\\rio_pro.npy',allow_pickle=True)

In [ ]:
X =[]
exclude =[]
for i in range(len(img_array)):
    if img_array[i].shape == (10000,):
        X.append(img_array[i])
    else:
        exclude.append(i)
X =np.array(X)
#also remove from the dataframe
df.drop(df.index[exclude],inplace=True)

In [ ]:

#Define a Convolutional Neural Network Model
inputs = tf.keras.Input(shape=(100, 100, 1))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 global_average_pooling2d (G  (None, 32)               0     

In [ ]:
save_at = "model_regression_disc.hdf5"
save_best2 = ModelCheckpoint (save_at, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='max')
#set up the x, y for training 
Y = np.array(df.stage.tolist())
#X_test = X[300:,]
#Y_test = Y[300:,]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20, random_state=13)
img_rows, img_cols = 100, 100
input_shape = (img_rows, img_cols, 1)
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
#X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

In [ ]:
history = model.fit( X_train, Y_train, 
                    epochs = 15, batch_size = 200, 
                    callbacks=[save_best2], verbose=1)

Epoch 1/15
166/166 [==============================] - 64s 374ms/step - loss: 1853740.5000
Epoch 2/15
 93/166 [===============>..............] - ETA: 26s - loss: 1493374.8750

In [ ]:
plt.figure(figsize=(6, 5))
# training loss
plt.plot(history.history['loss'], color='r')
#validation loss
plt.plot(history.history['val_loss'], color='g')
plt.show()
#125665

In [ ]:
Y_pred = (model.predict(X_val))
np.random.seed(23)
for rand_num in np.random.randint(0, 38, 5):
    plt.figure()
    plt.imshow(X_val[rand_num].reshape(100, 100),cmap='gray'), plt.axis('off')
    plt.title(str(Y_val[rand_num])+"<-real\n"+str(Y_pred[rand_num]) +"<-pred\n"+' stage', color='g')

In [ ]:
Y_pred

In [ ]:
Y_val

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_val, Y_pred)
#699581.0319818744